In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#같은 workload를 세번 실험 1, 2, 3 내용중 cnn만 뽑은 data = cnn_exp.pickle
all_data = pickle.load(open('./cnn_exp.pickle','rb'))
all_data.head()
#데이터 전체를 생략없이 보기위한 코드 
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', -1)

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv,exp_name,instance_name,dataset,model,optimizer,batchsize
tmp_index,,,,,,,,,,,,,,,,,,,,,
g3.4xlarge-mnist-vggsmall-SGD-16-20200726-1118012020_07_26_11_18_56.csv,0.000,30.880,39.487,121.439,375.868,19.584,25.568,6.720,3159.192,3.488,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,mnist,vggsmall,SGD,16
g3.4xlarge-cifar10-vggsmall-SGD-16-20200726-1136262020_07_26_11_37_24.csv,0.000,29.568,39.008,22.911,462.422,19.360,25.440,6.752,3307.216,3.872,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,vggsmall,SGD,16
g3.4xlarge-mnist-resnetsmall-SGD-64-20200726-1117042020_07_26_11_17_16.csv,48.541,29.440,38.784,224.636,344.057,30.238,28.224,6.720,13823.039,3.840,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,mnist,resnetsmall,SGD,64
g3.4xlarge-mnist-lenet5-SGD-64-20200726-1114532020_07_26_11_14_59.csv,0.000,29.374,38.848,43.998,87.901,24.735,28.478,6.720,6435.077,3.904,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,mnist,lenet5,SGD,64
g3.4xlarge-cifar10-lenet5-SGD-16-20200726-1132282020_07_26_11_32_43.csv,0.000,32.640,44.062,22.622,72.608,7.040,25.504,7.584,5024.138,4.448,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,16


In [3]:
#datasets : mnist,cifar10,fmnist 
#models : lenet5,vggsmall,resnetsmall
#batchsizes : 16,32,64,128,256
datasets = list(all_data['dataset'].value_counts().index)
models = list(all_data['model'].value_counts().index)
batchsizes = list(all_data['batchsize'].value_counts().index)

In [4]:
#사용자 지정 input_instance 와 target_instance정보만 골라서 가져오기 
def extracting(input_instance, target_instance):
    input_feature = all_data[all_data['instance_name'] == input_instance]
    target_feature = all_data[all_data['instance_name'] == target_instance]
    return input_feature , target_feature

In [5]:
# input = g3.4xlarge , target = g4dn.xlarge 로 설정하여 값 추출해오기
input_feature,target_feature=extracting('g3.4xlarge','g4dn.xlarge')

In [6]:
# input 값과 target 값 labeling 방법으로 처음 시도에서는 dataset, model, optimizer, batchsize 에 맞춰
# 두가지 dataframe 을 join 을 한 후 , 다시 나누는 과정을 거쳤지만 
# 그렇게 할 필요 없이, exp_name, dataset, model, optimizer, batchsize 순서로 정렬을 시키면 양쪽 dataframe 
# 짝이 맞춰저서 labeling 이 된다. 
sort_columns = ['exp_name', 'dataset','model', 'optimizer', 'batchsize']

In [7]:
#위의 sort_columns 를 기준으로 input_feature 와 target_feature 정렬하기 
sortInput_feature = input_feature.sort_values(sort_columns)
sortTarget_feature = target_feature.sort_values(sort_columns)

In [8]:
#정렬된 input_feature 
sortInput_feature

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv,exp_name,instance_name,dataset,model,optimizer,batchsize
tmp_index,,,,,,,,,,,,,,,,,,,,,
g3.4xlarge-cifar10-lenet5-SGD-128-20200726-1133342020_07_26_11_33_40.csv,0.0,29.568,38.912,21.503,143.582,24.864,28.704,6.720,4714.104,3.840,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,128
g3.4xlarge-cifar10-lenet5-SGD-16-20200726-1132282020_07_26_11_32_43.csv,0.0,32.640,44.062,22.622,72.608,7.040,25.504,7.584,5024.138,4.448,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,16
g3.4xlarge-cifar10-lenet5-SGD-256-20200726-1133492020_07_26_11_33_54.csv,0.0,30.496,38.718,23.775,213.244,25.344,28.767,6.880,4586.654,3.936,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,256
g3.4xlarge-cifar10-lenet5-SGD-32-20200726-1132562020_07_26_11_33_06.csv,0.0,30.111,38.686,20.288,78.238,25.215,28.416,6.688,5657.613,3.520,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,32
g3.4xlarge-cifar10-lenet5-SGD-64-20200726-1133172020_07_26_11_33_24.csv,0.0,29.248,38.975,20.992,99.805,24.768,28.448,6.656,5619.818,3.776,...,0.0,0.0,0.0,0.0,exp_01,g3.4xlarge,cifar10,lenet5,SGD,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g3.4xlarge-mnist-vggsmall-SGD-128-20200728-0229172020_07_28_02_29_52.csv,0.0,29.376,38.368,833.700,2087.912,42.177,28.544,6.784,4351.599,3.456,...,0.0,0.0,0.0,0.0,exp_03,g3.4xlarge,mnist,vggsmall,SGD,128
g3.4xlarge-mnist-vggsmall-SGD-16-20200728-0225532020_07_28_02_26_48.csv,0.0,30.688,38.720,119.425,371.360,18.464,25.152,6.848,3198.086,3.520,...,0.0,0.0,0.0,0.0,exp_03,g3.4xlarge,mnist,vggsmall,SGD,16
g3.4xlarge-mnist-vggsmall-SGD-256-20200728-0230102020_07_28_02_30_43.csv,0.0,29.664,38.304,1689.442,3945.635,49.792,28.384,7.136,4432.098,3.840,...,0.0,0.0,0.0,0.0,exp_03,g3.4xlarge,mnist,vggsmall,SGD,256


In [9]:
#정렬된 target_feature
sortTarget_feature

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv,exp_name,instance_name,dataset,model,optimizer,batchsize
tmp_index,,,,,,,,,,,,,,,,,,,,,
g4dn.xlarge-cifar10-lenet5-SGD-128-20200727-0226072020_07_27_02_26_13.csv,0.0,10.304,30.656,15.648,85.823,16.672,22.816,5.184,4032.860,2.368,...,0.0,0.0,0.0,0.0,exp_01,g4dn.xlarge,cifar10,lenet5,SGD,128
g4dn.xlarge-cifar10-lenet5-SGD-16-20200727-0225082020_07_27_02_25_21.csv,0.0,21.152,48.287,25.280,62.912,7.616,25.888,7.072,3141.886,3.008,...,0.0,0.0,0.0,0.0,exp_01,g4dn.xlarge,cifar10,lenet5,SGD,16
g4dn.xlarge-cifar10-lenet5-SGD-256-20200727-0226212020_07_27_02_26_26.csv,0.0,12.064,29.888,17.280,119.038,18.015,19.136,4.096,3568.577,3.104,...,0.0,0.0,0.0,0.0,exp_01,g4dn.xlarge,cifar10,lenet5,SGD,256
g4dn.xlarge-cifar10-lenet5-SGD-32-20200727-0225332020_07_27_02_25_42.csv,0.0,21.760,47.040,24.992,72.991,28.319,33.344,6.176,3589.570,4.096,...,0.0,0.0,0.0,0.0,exp_01,g4dn.xlarge,cifar10,lenet5,SGD,32
g4dn.xlarge-cifar10-lenet5-SGD-64-20200727-0225522020_07_27_02_25_59.csv,0.0,18.879,39.904,20.512,75.167,23.263,29.440,5.696,3891.328,3.840,...,0.0,0.0,0.0,0.0,exp_01,g4dn.xlarge,cifar10,lenet5,SGD,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g4dn.xlarge-mnist-vggsmall-SGD-128-20200728-0227432020_07_28_02_28_07.csv,0.0,18.079,41.120,469.851,1164.660,44.800,30.112,7.328,4066.685,3.360,...,0.0,0.0,0.0,0.0,exp_03,g4dn.xlarge,mnist,vggsmall,SGD,128
g4dn.xlarge-mnist-vggsmall-SGD-16-20200728-0225222020_07_28_02_25_58.csv,0.0,15.232,33.376,44.736,214.560,15.104,21.792,4.192,2455.279,3.488,...,0.0,0.0,0.0,0.0,exp_03,g4dn.xlarge,mnist,vggsmall,SGD,16
g4dn.xlarge-mnist-vggsmall-SGD-256-20200728-0228222020_07_28_02_28_45.csv,0.0,29.984,43.167,932.695,2148.078,54.238,31.872,7.520,4905.492,4.096,...,0.0,0.0,0.0,0.0,exp_03,g4dn.xlarge,mnist,vggsmall,SGD,256


In [10]:
# cnn 3개 모델과 3가지 dataset중 각각 1가지의 test model 과 dataset 골라서 값 추출하기
def testInput_selecting(model,dataset):
    test_condition = ((sortInput_feature['model'] == model) &
                      (sortInput_feature['dataset'] == dataset))
    # test condition 에 반대되는 값들은 training condition 
    training_input=sortInput_feature[~test_condition]
    test_input=sortInput_feature[test_condition]
    return training_input, test_input

In [11]:
def testTarget_selecting(model,dataset):
    test_condition = ((sortTarget_feature['model'] == model) &
              (sortTarget_feature['dataset'] == dataset))
    training_target = sortTarget_feature[~test_condition]
    test_target=sortTarget_feature[test_condition]
    return training_target,test_target

In [12]:
# 사용자 지정 model 과 dataset 에 따라 training data 와 test data가 나누어지는 과정 
training_input,test_input=testInput_selecting('vggsmall','fmnist')
training_target,test_target=testTarget_selecting('vggsmall','fmnist')

In [13]:
# 전체 columns 에서 학습에 사용되지 않는 columns 들 drop 하기위한 list 생성 
drop_columns = sort_columns + ['instance_name']
#학습에 쓰이지 않는 columns drop
training_input=training_input.drop(drop_columns,axis=1)
training_target=training_target.drop(drop_columns,axis=1)
test_input=test_input.drop(drop_columns,axis=1)
test_target = test_target.drop(drop_columns,axis=1)

In [14]:
#전체 :cnn 3가지 * dataset 3가지 * batchsize 5가지 * 실험 3번 = 135 rows
# test data = vggsmall & fmnist : cnn 1가지 * dataset 1가지 * batchsize 5가지 * 3번의 실험 = 15 rows
# traing data = 전체 - test data = 120 rows 
training_input

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_GreaterEqual,Host_Mean,Host_Maximum,Host_RandomUniform,Host_AssignVariableOp,Host_FusedBatchNormGradV3,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv
tmp_index,,,,,,,,,,,,,,,,,,,,,
g3.4xlarge-cifar10-lenet5-SGD-128-20200726-1133342020_07_26_11_33_40.csv,0.0,29.568,38.912,21.503,143.582,24.864,28.704,6.720,4714.104,3.840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-cifar10-lenet5-SGD-16-20200726-1132282020_07_26_11_32_43.csv,0.0,32.640,44.062,22.622,72.608,7.040,25.504,7.584,5024.138,4.448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-cifar10-lenet5-SGD-256-20200726-1133492020_07_26_11_33_54.csv,0.0,30.496,38.718,23.775,213.244,25.344,28.767,6.880,4586.654,3.936,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-cifar10-lenet5-SGD-32-20200726-1132562020_07_26_11_33_06.csv,0.0,30.111,38.686,20.288,78.238,25.215,28.416,6.688,5657.613,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-cifar10-lenet5-SGD-64-20200726-1133172020_07_26_11_33_24.csv,0.0,29.248,38.975,20.992,99.805,24.768,28.448,6.656,5619.818,3.776,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g3.4xlarge-mnist-vggsmall-SGD-128-20200728-0229172020_07_28_02_29_52.csv,0.0,29.376,38.368,833.700,2087.912,42.177,28.544,6.784,4351.599,3.456,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-mnist-vggsmall-SGD-16-20200728-0225532020_07_28_02_26_48.csv,0.0,30.688,38.720,119.425,371.360,18.464,25.152,6.848,3198.086,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-mnist-vggsmall-SGD-256-20200728-0230102020_07_28_02_30_43.csv,0.0,29.664,38.304,1689.442,3945.635,49.792,28.384,7.136,4432.098,3.840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
training_target

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_GreaterEqual,Host_Mean,Host_Maximum,Host_RandomUniform,Host_AssignVariableOp,Host_FusedBatchNormGradV3,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv
tmp_index,,,,,,,,,,,,,,,,,,,,,
g4dn.xlarge-cifar10-lenet5-SGD-128-20200727-0226072020_07_27_02_26_13.csv,0.0,10.304,30.656,15.648,85.823,16.672,22.816,5.184,4032.860,2.368,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-cifar10-lenet5-SGD-16-20200727-0225082020_07_27_02_25_21.csv,0.0,21.152,48.287,25.280,62.912,7.616,25.888,7.072,3141.886,3.008,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-cifar10-lenet5-SGD-256-20200727-0226212020_07_27_02_26_26.csv,0.0,12.064,29.888,17.280,119.038,18.015,19.136,4.096,3568.577,3.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-cifar10-lenet5-SGD-32-20200727-0225332020_07_27_02_25_42.csv,0.0,21.760,47.040,24.992,72.991,28.319,33.344,6.176,3589.570,4.096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-cifar10-lenet5-SGD-64-20200727-0225522020_07_27_02_25_59.csv,0.0,18.879,39.904,20.512,75.167,23.263,29.440,5.696,3891.328,3.840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g4dn.xlarge-mnist-vggsmall-SGD-128-20200728-0227432020_07_28_02_28_07.csv,0.0,18.079,41.120,469.851,1164.660,44.800,30.112,7.328,4066.685,3.360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-mnist-vggsmall-SGD-16-20200728-0225222020_07_28_02_25_58.csv,0.0,15.232,33.376,44.736,214.560,15.104,21.792,4.192,2455.279,3.488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-mnist-vggsmall-SGD-256-20200728-0228222020_07_28_02_28_45.csv,0.0,29.984,43.167,932.695,2148.078,54.238,31.872,7.520,4905.492,4.096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# test = vggsmall & fmnist = 15 rows
test_input

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_GreaterEqual,Host_Mean,Host_Maximum,Host_RandomUniform,Host_AssignVariableOp,Host_FusedBatchNormGradV3,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv
tmp_index,,,,,,,,,,,,,,,,,,,,,
g3.4xlarge-fmnist-vggsmall-SGD-128-20200726-1130342020_07_26_11_31_08.csv,0.0,30.400,39.199,838.865,2088.668,42.814,28.768,7.104,4435.401,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-16-20200726-1127072020_07_26_11_28_02.csv,0.0,30.528,39.391,119.997,364.606,19.647,25.215,6.912,3434.256,4.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-256-20200726-1131272020_07_26_11_32_00.csv,0.0,29.792,39.103,1697.025,3944.822,53.726,28.832,6.976,5209.407,3.488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-32-20200726-1128292020_07_26_11_29_13.csv,0.0,31.456,38.815,223.677,613.844,19.776,25.408,6.784,3630.691,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-64-20200726-1129352020_07_26_11_30_13.csv,0.0,29.056,38.975,422.232,1148.139,39.840,28.736,6.752,4604.166,3.872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-128-20200727-0459192020_07_27_04_59_53.csv,0.0,28.736,38.368,832.796,2089.333,42.752,28.896,6.816,4639.778,3.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-16-20200727-0455532020_07_27_04_56_48.csv,0.0,30.432,38.176,119.520,374.492,18.528,25.088,6.783,3241.771,3.776,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-256-20200727-0500122020_07_27_05_00_44.csv,0.0,30.912,38.720,1684.471,3952.970,51.520,28.415,7.040,4875.399,3.808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g3.4xlarge-fmnist-vggsmall-SGD-32-20200727-0457152020_07_27_04_57_58.csv,0.0,29.984,38.336,225.085,603.228,19.936,25.312,6.656,3336.975,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
test_target

,Device_AddN,Device_ArgMax,Device_AssignAddVariableOp,Device_BiasAdd,Device_BiasAddGrad,Device_Cast,Device_DivNoNan,Device_Equal,Device_IDLE,Device_LogicalAnd,...,Host_GreaterEqual,Host_Mean,Host_Maximum,Host_RandomUniform,Host_AssignVariableOp,Host_FusedBatchNormGradV3,Host_FusedBatchNormV3,Host_RealDiv,Host_Relu,Host_FloorDiv
tmp_index,,,,,,,,,,,,,,,,,,,,,
g4dn.xlarge-fmnist-vggsmall-SGD-128-20200727-0223082020_07_27_02_23_35.csv,0.0,21.952,47.296,489.083,1224.657,52.256,36.352,7.808,3126.622,3.968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-16-20200727-0220422020_07_27_02_21_19.csv,0.0,17.760,39.935,52.384,247.102,15.520,23.071,4.864,2439.427,3.584,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-256-20200727-0223502020_07_27_02_24_16.csv,0.0,28.896,49.088,953.942,2237.959,61.822,35.872,8.192,3822.489,4.096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-32-20200727-0221402020_07_27_02_22_09.csv,0.0,18.431,41.248,132.766,389.180,19.680,28.000,7.840,2543.270,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-64-20200727-0222262020_07_27_02_22_53.csv,0.0,19.296,44.192,250.301,715.386,45.088,33.120,6.720,3212.567,2.688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-128-20200727-0455162020_07_27_04_55_39.csv,0.0,22.912,39.871,464.506,1185.268,43.903,29.408,7.296,3399.263,2.368,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-16-20200727-0452562020_07_27_04_53_32.csv,0.0,14.304,33.376,45.440,230.045,16.000,21.855,5.952,2438.340,2.048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-256-20200727-0455532020_07_27_04_56_16.csv,0.0,20.672,42.048,931.191,2122.730,54.174,30.880,6.752,3709.624,3.552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g4dn.xlarge-fmnist-vggsmall-SGD-32-20200727-0453512020_07_27_04_54_20.csv,0.0,16.384,37.120,125.247,357.023,19.168,21.664,6.464,2313.131,2.816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training Model 

# RandomForest

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
clf = RandomForestRegressor().fit(training_input, training_target)
y_pred = clf.predict(test_input)
y_true = test_target.values

In [20]:
predict=pd.DataFrame(y_pred)

In [21]:
pickle.dump(predict, open('./randomForest_g4dn.xlarge_prediction.pickle', 'wb'))

In [ ]:
real=pd.DataFrame(y_true)

In [ ]:
pickle.dump(real, open('./randomForest_g4dn.xlarge_real.pickle', 'wb'))

# Linear Regression

In [ ]:
#Linear Regression for Multioutput Regression
from sklearn.linear_model import LinearRegression

In [ ]:
linear_model = LinearRegression()
linear_model.fit(training_input,training_target)

In [ ]:
pred=pd.DataFrame(linear_model.predict(test_input))

In [ ]:
pickle.dump(pred, open('./linearRegression_g4dn.xlarge_prediction.pickle', 'wb'))

In [ ]:
real=pd.DataFrame(test_target)

In [ ]:
pickle.dump(real, open('./linearRegression_g4dn.xlarge_real.pickle', 'wb'))

# K-Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


In [ ]:
knnModel = KNeighborsRegressor()
knnModel.fit(training_input,training_target)

In [ ]:
pred=pd.DataFrame(knnModel.predict(test_input))

In [ ]:
pred

In [ ]:
pickle.dump(pred, open('./knn_g4dn.xlarge_prediction.pickle', 'wb'))

# DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dcTree=DecisionTreeRegressor()
dcTree.fit(training_input,training_target)

In [ ]:
pred=pd.DataFrame(dcTree.predict(test_input))

In [ ]:
pred

In [ ]:
pickle.dump(pred,open('./decisionTree_g4dn.xlarge_prediction.pickle','wb'))